# Temperature Exploration

August 8, 2022  
Gully & Ryan H.

Using Gollum to explore different stars and their temperatures.

In [ ]:
from gollum.phoenix import PHOENIXGrid, PHOENIXSpectrum
from specutils import Spectrum1D
import pandas as pd
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy.modeling import physical_models

### Compute Starspot Contrast

Example from a sunspot

In [ ]:
kep_response = pd.read_csv('../../data/kepler_response_hires1.txt',
                           names=['wl_nm','trans'],
                           skiprows=9, delim_whitespace=True)
kep_response['wl_ang'] = kep_response.wl_nm * 10.0
kep_response = kep_response[(kep_response.wl_ang > 4200) & (kep_response.wl_ang < 9200)]
kep_response['trans_norm'] = kep_response.trans / np.sum(kep_response.trans)
kep_response = kep_response.reset_index(drop=True)

In [ ]:
tess_response = pd.read_csv('../../data/tess-response-function-v1.0.csv', skiprows=8, names=['wl_nm', 'trans'])

## Create heat maps

In [ ]:
columns = [_ for _ in range(2300, 7100, 100)]
index = [_ for _ in range(7000, 2200, -100)]

### Kepler heat map

In [ ]:
Kepler_ratios = np.full((48,48), np.nan)

In [ ]:
wls = np.arange(5000, 7000, 500)*u.Angstrom
for row in range(len(Kepler_ratios)): # Row
    for col in range(len(Kepler_ratios)): # Column
        if (col>=len(Kepler_ratios)-row-1):
            bb_cool = physical_models.BlackBody(temperature = (2300 + col*100)*u.Kelvin)
            bb_hot =  physical_models.BlackBody(temperature = (7000 - row*100)*u.Kelvin)
            contrast = bb_hot(wls)/bb_cool(wls)
            mean = contrast.mean()
            Kepler_ratios[row][col] = mean

In [ ]:
# convert array into dataframe
Kepler_ratios_df = pd.DataFrame(Kepler_ratios, columns=columns, index=index)

# save the dataframe as a csv file
Kepler_ratios_df.to_csv("../../data/Black Body/Kepler_ratios.csv")

In [ ]:
ax = plt.subplot()
im = ax.imshow(Kepler_ratios, origin='upper', extent=(2300, 7000, 2300, 7000), aspect=1)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size='5%', pad=0.05)
plt.colorbar(im, cax=cax)

x = np.linspace(2300, 6300, 100)
y = -1.1668835503435e-4*x**2 + 1.374977439464*x - 567.8141350083
ax.plot(x, y, color='red', linestyle='dashed', label='Fang et. al. 2015')

x = np.linspace(2300, 3300, 100)
y = 0.86*x
ax.plot(x, y, color='white', linestyle='dashed', label='Rackham 2018')

x = np.linspace(3880, 6300, 100)
y = 0.418*x + 1620
ax.plot(x, y, color='black', linestyle='dashed', label='Rackham 2019')

ax.set_xlim(2300, 6300)
ax.set_ylim(2300, 6300)

ax.legend(loc='upper left')
ax.set_xlabel('$T_{\mathrm{phot}}$ (K)')
ax.set_ylabel('$T_{\mathrm{spot}}$ (K)')
ax.set_title('Kepler $T_{\mathrm{phot}}$ vs. $T_{\mathrm{spot}}$ (K)')

plt.savefig('../../papers/paper1/figures/Black Body Contrast/Kepler_contrast.png', bbox_inches='tight', dpi=300, facecolor='white', transparent=False)

### TESS heat map

In [ ]:
TESS_ratios = np.full((48,48), np.nan)

In [ ]:
# wls = np.arange(4540, 11292, 500)*u.Angstrom
wls = np.arange(8000, 10000, 500)*u.Angstrom
for row in range(len(TESS_ratios)): # Row
    for col in range(len(TESS_ratios)): # Column
        if (col>=len(TESS_ratios)-row-1):
            bb_cool = physical_models.BlackBody(temperature = (2300 + col*100)*u.Kelvin)
            bb_hot =  physical_models.BlackBody(temperature = (7000 - row*100)*u.Kelvin)
            contrast = bb_hot(wls)/bb_cool(wls)
            mean = contrast.mean()
            TESS_ratios[row][col] = mean

In [ ]:
# convert array into dataframe
TESS_ratios_df = pd.DataFrame(TESS_ratios, columns=columns, index=index)

# save the dataframe as a csv file
TESS_ratios_df.to_csv("../../data/Black Body/TESS_ratios.csv", index=False, header=False)

In [ ]:
ax = plt.subplot()
im = ax.imshow(TESS_ratios, origin='upper', extent=(2300, 7000, 2300, 7000), aspect=1)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size='5%', pad=0.05)
plt.colorbar(im, cax=cax)

x = np.linspace(2300, 6300, 100)
y = -1.1668835503435e-4*x**2 + 1.374977439464*x - 567.8141350083
ax.plot(x, y, color='red', linestyle='dashed', label='Fang et. al. 2015')

x = np.linspace(2300, 3300, 100)
y = 0.86*x
ax.plot(x, y, color='white', linestyle='dashed', label='Rackham 2018')

x = np.linspace(3880, 6300, 100)
y = 0.418*x + 1620
ax.plot(x, y, color='black', linestyle='dashed', label='Rackham 2019')

ax.set_xlim(2300, 6300)
ax.set_ylim(2300, 6300)

ax.legend(loc='upper left')
ax.set_xlabel('$T_{\mathrm{phot}}$ (K)')
ax.set_ylabel('$T_{\mathrm{spot}}$ (K)')
ax.set_title('TESS $T_{\mathrm{phot}}$ vs. $T_{\mathrm{spot}}$ (K)')

plt.savefig('../../papers/paper1/figures/Black Body Contrast/TESS_contrast.png', bbox_inches='tight', dpi=300, facecolor='white', transparent=False)

### Ratio of TESS over Kepler

In [ ]:
TESS_v_Kepler_ratios = np.full((48,48), np.nan)

In [ ]:
for row in range(len(TESS_v_Kepler_ratios)): # Row
    for col in range(len(TESS_v_Kepler_ratios)): # Column
        if (col>=len(TESS_v_Kepler_ratios)-row-1):
            TESS_v_Kepler_ratios[row][col] = TESS_ratios[row][col]/Kepler_ratios[row][col]

In [ ]:
# convert array into dataframe
DF = pd.DataFrame(TESS_v_Kepler_ratios)
  
# save the dataframe as a csv file
DF.to_csv("../../data/Black Body/TESS_v_Kepler_ratios.csv", index=False, header=False)

In [ ]:
ax = plt.subplot()
im = ax.imshow(TESS_v_Kepler_ratios, origin='upper', extent=(2300, 7000, 2300, 7000), aspect=1)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size='5%', pad=0.05)
plt.colorbar(im, cax=cax)

x = np.linspace(2300, 6300, 100)
y = -1.1668835503435e-4*x**2 + 1.374977439464*x - 567.8141350083
ax.plot(x, y, color='red', linestyle='dashed', label='Fang et. al. 2015')

x = np.linspace(2300, 3300, 100)
y = 0.86*x
ax.plot(x, y, color='white', linestyle='dashed', label='Rackham 2018')

x = np.linspace(3880, 6300, 100)
y = 0.418*x + 1620
ax.plot(x, y, color='black', linestyle='dashed', label='Rackham 2019')

ax.set_xlim(2300, 6300)
ax.set_ylim(2300, 6300)

ax.legend(loc='upper left')
ax.set_xlabel('$T_{\mathrm{phot}}$ (K)')
ax.set_ylabel('$T_{\mathrm{spot}}$ (K)')
ax.set_title('Ratio of TESS over Kepler')

plt.savefig('../../papers/paper1/figures/Black Body Contrast/TESS_v_Kepler_ratios.png', bbox_inches='tight', dpi=300, facecolor='white', transparent=False)

### Ratio of 1-TESS over 1-Kepler

In [ ]:
one_minus_ratio = np.full((48,48), np.nan)

In [ ]:
Kepler_ratios = pd.read_csv('../../data/Black Body/Kepler_ratios.csv', names=columns)
TESS_ratios = pd.read_csv('../../data/Black Body/TESS_ratios.csv', names=columns)

In [ ]:
if type(Kepler_ratios) == pd.core.frame.DataFrame:
    Kepler_ratios = Kepler_ratios.to_numpy()
if type(TESS_ratios) == pd.core.frame.DataFrame:
    TESS_ratios = TESS_ratios.to_numpy()

In [ ]:
for row in range(len(one_minus_ratio)): # Row
    for col in range(len(one_minus_ratio)): # Column
        if (col>=len(one_minus_ratio)-row-1):
            if 1-TESS_ratios[row][col] == 0 and 1-Kepler_ratios[row][col] == 0:
                one_minus_ratio[row][col] = np.NaN
            else:
                one_minus_ratio[row][col] = (1-TESS_ratios[row][col])/(1-Kepler_ratios[row][col])

In [ ]:
# convert array into dataframe
DF = pd.DataFrame(one_minus_ratio)
  
# save the dataframe as a csv file
DF.to_csv("../../data/Black Body/one_minus_ratio.csv", index=False, header=False)

In [ ]:
ax = plt.subplot()
im = ax.imshow(one_minus_ratio, origin='upper', extent=(2300, 7000, 2300, 7000), aspect=1, vmin=0, vmax=1)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size='5%', pad=0.05)
plt.colorbar(im, cax=cax)

x = np.linspace(2300, 6300, 100)
y = -1.1668835503435e-4*x**2 + 1.374977439464*x - 567.8141350083
ax.plot(x, y, color='red', linestyle='dashed', label='Fang et. al. 2015')

x = np.linspace(2300, 3300, 100)
y = 0.86*x
ax.plot(x, y, color='white', linestyle='dashed', label='Rackham 2018')

x = np.linspace(3880, 6300, 100)
y = 0.418*x + 1620
ax.plot(x, y, color='black', linestyle='dashed', label='Rackham 2019')

ax.set_xlim(2300, 6300)
ax.set_ylim(2300, 6300)

ax.legend(loc='upper left')
ax.set_xlabel('$T_{\mathrm{phot}}$ (K)')
ax.set_ylabel('$T_{\mathrm{spot}}$ (K)')
ax.set_title('Ratio of 1-TESS over 1-Kepler')

plt.savefig('../../papers/paper1/figures/Black Body Contrast/one_minus_ratio.png', bbox_inches='tight', dpi=300, facecolor='white', transparent=False)

## Generate graphs with CSV's

In [ ]:
Kepler_ratios = pd.read_csv('../../data/Kepler_ratios.csv')
TESS_ratios = pd.read_csv('../../data/TESS_ratios.csv')
TESS_v_Kepler_ratios = pd.read_csv('../../data/TESS_v_Kepler_ratios.csv')
one_minus_ratio = pd.read_csv('../../data/one_minus_ratio.csv')

### Kepler

In [ ]:
ax = plt.subplot()
im = ax.imshow(Kepler_ratios, origin='upper', extent=(2300, 7000, 2300, 7000), aspect=1)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size='5%', pad=0.05)
plt.colorbar(im, cax=cax)

x = np.linspace(2300, 6300, 100)
y = -1.1668835503435e-4*x**2 + 1.374977439464*x - 567.8141350083
ax.plot(x, y, color='red', linestyle='dashed', label='Fang et. al. 2015')

x = np.linspace(2300, 3300, 100)
y = 0.86*x
ax.plot(x, y, color='white', linestyle='dashed', label='Rackham 2018')

x = np.linspace(3880, 6300, 100)
y = 0.418*x + 1620
ax.plot(x, y, color='black', linestyle='dashed', label='Rackham 2019')

ax.set_xlim(2300, 6300)
ax.set_ylim(2300, 6300)

ax.legend(loc='upper left')
ax.set_xlabel('$T_{\mathrm{phot}}$ (K)')
ax.set_ylabel('$T_{\mathrm{spot}}$ (K)')
ax.set_title('Kepler $T_{\mathrm{phot}}$ vs. $T_{\mathrm{spot}}$ (K)')

# cax.set_title('Contrast', rotation=270)

plt.savefig('../../papers/paper1/figures/Black Body Contrast/Kepler_contrast.png', bbox_inches='tight', dpi=300, facecolor='white', transparent=False)

### TESS

In [ ]:
ax = plt.subplot()
im = ax.imshow(TESS_ratios, origin='upper', extent=(2300, 7000, 2300, 7000), aspect=1)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size='5%', pad=0.05)
plt.colorbar(im, cax=cax)

x = np.linspace(2300, 6300, 100)
y = -1.1668835503435e-4*x**2 + 1.374977439464*x - 567.8141350083
ax.plot(x, y, color='red', linestyle='dashed', label='Fang et. al. 2015')

x = np.linspace(2300, 3300, 100)
y = 0.86*x
ax.plot(x, y, color='white', linestyle='dashed', label='Rackham 2018')

x = np.linspace(3880, 6300, 100)
y = 0.418*x + 1620
ax.plot(x, y, color='black', linestyle='dashed', label='Rackham 2019')

ax.set_xlim(2300, 6300)
ax.set_ylim(2300, 6300)

ax.legend(loc='upper left')
ax.set_xlabel('$T_{\mathrm{phot}}$ (K)')
ax.set_ylabel('$T_{\mathrm{spot}}$ (K)')
ax.set_title('TESS $T_{\mathrm{phot}}$ vs. $T_{\mathrm{spot}}$ (K)')

# cax.set_title('Contrast', rotation=270)

plt.savefig('../../papers/paper1/figures/Black Body Contrast/TESS_contrast.png', bbox_inches='tight', dpi=300, facecolor='white', transparent=False)

### Ratio of TESS over Kepler

In [ ]:
ax = plt.subplot()
im = ax.imshow(TESS_v_Kepler_ratios, origin='upper', extent=(2300, 7000, 2300, 7000), aspect=1)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size='5%', pad=0.05)
plt.colorbar(im, cax=cax)

x = np.linspace(2300, 6300, 100)
y = -1.1668835503435e-4*x**2 + 1.374977439464*x - 567.8141350083
ax.plot(x, y, color='red', linestyle='dashed', label='Fang et. al. 2015')

x = np.linspace(2300, 3300, 100)
y = 0.86*x
ax.plot(x, y, color='white', linestyle='dashed', label='Rackham 2018')

x = np.linspace(3880, 6300, 100)
y = 0.418*x + 1620
ax.plot(x, y, color='black', linestyle='dashed', label='Rackham 2019')

ax.set_xlim(2300, 6300)
ax.set_ylim(2300, 6300)

ax.legend(loc='upper left')
ax.set_xlabel('$T_{\mathrm{phot}}$ (K)')
ax.set_ylabel('$T_{\mathrm{spot}}$ (K)')
ax.set_title('Ratio of TESS over Kepler')

plt.savefig('../../papers/paper1/figures/Black Body Contrast/TESS_v_Kepler_ratios.png', bbox_inches='tight', dpi=300, facecolor='white', transparent=False)

### Ratio of 1-TESS over 1-Kepler

In [ ]:
ax = plt.subplot()
im = ax.imshow(one_minus_ratio, origin='upper', extent=(2300, 7000, 2300, 7000), aspect=1, vmin=0, vmax=1)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size='5%', pad=0.05)
plt.colorbar(im, cax=cax)

x = np.linspace(2300, 6300, 100)
y = -1.1668835503435e-4*x**2 + 1.374977439464*x - 567.8141350083
ax.plot(x, y, color='red', linestyle='dashed', label='Fang et. al. 2015')

x = np.linspace(2300, 3300, 100)
y = 0.86*x
ax.plot(x, y, color='white', linestyle='dashed', label='Rackham 2018')

x = np.linspace(3880, 6300, 100)
y = 0.418*x + 1620
ax.plot(x, y, color='black', linestyle='dashed', label='Rackham 2019')

ax.set_xlim(2300, 6300)
ax.set_ylim(2300, 6300)

ax.legend(loc='upper left')
ax.set_xlabel('$T_{\mathrm{phot}}$ (K)')
ax.set_ylabel('$T_{\mathrm{spot}}$ (K)')
ax.set_title('Ratio of 1-TESS over 1-Kepler')

plt.savefig('../../papers/paper1/figures/Black Body Contrast/one_minus_ratio.png', bbox_inches='tight', dpi=300, facecolor='white', transparent=False)